In [29]:
import numpy as np
from scipy.io import loadmat
import cvxopt

# DATA

In [30]:
mnist = loadmat("mnist")
Train_Data = mnist['train']
Test_Data = mnist['test']

In [31]:
print(f'shape of train data  : {Train_Data.shape}')
print(f'shape of test data  : {Test_Data.shape}')

shape of train data  : (8000, 785)
shape of test data  : (2000, 785)


In [32]:
# spliting Data to  input and label 
X_train = Train_Data[:, :-1]
y_train = Train_Data[:, -1]
X_test = Test_Data[:, :-1]
y_test = Test_Data[:, -1]

In [33]:
# function for changing labels to 1 and -1
def BinaryLabel(arr, turn_to_one):
    return np.where(arr == turn_to_one, 1, -1)

In [34]:
# preparing data for labels 3 and 8 

Train_mask = (y_train == 3) | (y_train == 8)
X_trn = X_train[Train_mask]
y_trn = BinaryLabel(y_train[Train_mask], 3)


Test_mask = (y_test == 3) | (y_test == 8)
X_tst = X_test[Test_mask]
y_tst = BinaryLabel(y_test[Test_mask], 3)

# Part A

In [35]:
class HardSVM(object):
    def __init__(self):
        
        self.alpha = None
        self.SV = None
        self.SV_label = None
  

    def fit(self, X, y):
        n_samples, n_features = X.shape
        K = np.array([[np.dot(X[i], X[j]) for j in range(n_samples)] for i in range(n_samples)])
        P = 1.0 * cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = 1.0 * cvxopt.matrix(y, (1,n_samples))
        b = cvxopt.matrix(0.0)
        G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
        h = cvxopt.matrix(np.zeros(n_samples))
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        alpha = np.ravel(solution['x'])
        SV = alpha > 1e-5
        self.alpha = alpha[SV]
        self.SV = X[SV]
        self.SV_label = y[SV]
        
        
        self.w = np.zeros(n_features)
        for n in range(len(self.alpha)):
            self.w += self.alpha[n] * self.SV_label[n] * self.SV[n]
            
            
        self.b = self.SV_label[0] - np.dot(self.SV[0], self.w)


    def predict(self, X):
        res = np.dot(X, self.w) + self.b
        
        return np.sign(res)

In [36]:
hard_model = HardSVM()

In [37]:
hard_model.fit(X_trn, y_trn)

     pcost       dcost       gap    pres   dres
 0: -2.3883e+02 -6.9284e+02  9e+03  8e+01  3e+00
 1: -4.3881e+02 -6.9632e+02  5e+03  3e+01  1e+00
 2: -2.0113e+02 -2.0273e+02  1e+03  1e+01  3e-01
 3: -7.7976e+01 -8.8452e+01  4e+02  3e+00  9e-02
 4: -4.0433e+01 -5.6017e+01  1e+02  6e-01  2e-02
 5: -3.2106e+01 -4.1542e+01  2e+01  9e-02  3e-03
 6: -3.4439e+01 -3.7847e+01  6e+00  2e-02  7e-04
 7: -3.5888e+01 -3.7073e+01  2e+00  4e-03  1e-04
 8: -3.6589e+01 -3.6828e+01  3e-01  2e-04  8e-06
 9: -3.6780e+01 -3.6788e+01  8e-03  4e-06  1e-07
10: -3.6786e+01 -3.6786e+01  1e-04  7e-08  2e-09
11: -3.6786e+01 -3.6786e+01  2e-06  7e-10  2e-11
Optimal solution found.


In [38]:
predictions = hard_model.predict(X_trn)
accuracy = sum(predictions == y_trn) / len(y_trn)
print(f" model Accuracy = {accuracy:.2f} on 'train' data")

 model Accuracy = 1.00 on 'train' data


In [39]:
predictions = hard_model.predict(X_tst)
accuracy = sum(predictions == y_tst) / len(y_tst)
print(f" model Accuracy = {accuracy:.2f} on 'test' data")

 model Accuracy = 0.94 on 'test' data


# Part B

In [41]:
class SoftSVM(object):
    def __init__(self, C=5):
        
        self.C = C
        self.alpha = None
        self.SV = None
        self.SV_label = None
  

    def fit(self, X, y):
        n_samples, n_features = X.shape
        K = np.array([[np.dot(X[i], X[j]) for j in range(n_samples)] for i in range(n_samples)])
        P = 1.0 * cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = 1.0 * cvxopt.matrix(y, (1,n_samples))
        b = cvxopt.matrix(0.0)
        G = cvxopt.matrix(np.vstack((np.diag(np.ones(n_samples) * -1), np.identity(n_samples))))
        h = cvxopt.matrix(np.hstack((np.zeros(n_samples), np.ones(n_samples) * self.C)))
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        alpha = np.ravel(solution['x'])
        SV = alpha > 1e-5
        self.alpha = alpha[SV]
        self.SV = X[SV]
        self.SV_label = y[SV]
        
        
        self.w = np.zeros(n_features)
        for n in range(len(self.alpha)):
            self.w += self.alpha[n] * self.SV_label[n] * self.SV[n]
            
        ind = np.arange(len(alpha))[SV]
        self.b = 0
        for n in range(len(self.alpha)):
            self.b += self.SV_label[n]
            self.b -= np.sum(self.alpha * self.SV_label * K[ind[n],SV])
        self.b /= len(self.alpha)


    def predict(self, X):
        res = np.dot(X, self.w) + self.b
        
        return np.sign(res)

In [13]:
soft_model = SoftSVM()

In [14]:
soft_model.fit(X_trn, y_trn)

     pcost       dcost       gap    pres   dres
 0: -6.2113e+02 -3.6754e+04  1e+05  1e+00  3e-12
 1: -3.1326e+02 -1.5605e+04  3e+04  2e-01  2e-12
 2: -7.2251e+01 -4.7610e+03  8e+03  6e-02  1e-12
 3: -3.9784e+00 -2.3315e+03  4e+03  2e-02  6e-13
 4:  4.2958e+01 -9.9023e+02  1e+03  6e-03  4e-13
 5:  4.4927e+00 -1.2475e+02  1e+02  7e-05  3e-13
 6: -1.8783e+01 -6.6760e+01  5e+01  2e-05  2e-13
 7: -2.7895e+01 -5.3652e+01  3e+01  4e-06  2e-13
 8: -3.2776e+01 -4.5085e+01  1e+01  3e-15  2e-13
 9: -3.5504e+01 -3.9212e+01  4e+00  5e-16  2e-13
10: -3.6481e+01 -3.7262e+01  8e-01  2e-15  2e-13
11: -3.6773e+01 -3.6805e+01  3e-02  7e-15  2e-13
12: -3.6786e+01 -3.6787e+01  5e-04  4e-15  2e-13
13: -3.6786e+01 -3.6786e+01  6e-06  3e-15  2e-13
Optimal solution found.


In [15]:
predictions = soft_model.predict(X_trn)
accuracy = sum(predictions == y_trn) / len(y_trn)
print(f" model Accuracy = {accuracy:.2f} on 'train' data")

 model Accuracy = 1.00 on 'train' data


In [16]:
predictions = soft_model.predict(X_tst)
accuracy = sum(predictions == y_tst) / len(y_tst)
print(f" model Accuracy = {accuracy:.2f} on 'test' data")

 model Accuracy = 0.94 on 'test' data


In [17]:
from sklearn.model_selection import KFold

best_c = 1
best_accuracy = 0
accuracy_dict = {}

for C in [0.0001, 0.001, 0.01, 0.1, 10, 100, 1000, 10000]:
    print(f"C is {C}")
    
    model = SoftSVM(C=C)
    
    
    num_folds = 5
    
    X = X_trn
    y = y_trn


    kfold = KFold(n_splits=num_folds)

   
    accuracies = []
        
    fold_num = 0
    for train, test in kfold.split(X):
        fold_num += 1
        print(f"fold num = {fold_num}")

        CV_X_train, CV_X_test, CV_y_train, CV_y_test = X[train], X[test], y[train], y[test]
        
        model.fit(CV_X_train, CV_y_train)
        
        predictions = model.predict(CV_X_test)
        accuracy = sum(predictions == CV_y_test) / len(CV_y_test)
        accuracies.append(accuracy)
        
        

    avg_accuracy = np.mean(accuracies)
    
    print(f"for C = {C} avarage accuracy = {avg_accuracy}")
    
    accuracy_dict[f"for C = {C} avarage accuracy"] = avg_accuracy
    
    if avg_accuracy > best_accuracy :
        best_accuracy = avg_accuracy
        best_c = C
        


C is 0.0001
fold num = 1
     pcost       dcost       gap    pres   dres
 0: -8.8431e+01 -4.0362e+00  9e+03  1e+02  3e-13
 1: -8.3665e+00 -1.5119e+00  5e+02  6e+00  3e-13
 2: -9.4003e-01 -4.8882e-01  3e+01  3e-01  3e-14
 3: -2.0233e-01 -3.4359e-01  2e+00  2e-02  3e-15
 4: -8.7017e-02 -2.7544e-01  2e-01  3e-18  2e-15
 5: -9.4164e-02 -1.1461e-01  2e-02  7e-19  8e-16
 6: -9.9153e-02 -1.0610e-01  7e-03  2e-18  5e-16
 7: -1.0023e-01 -1.0449e-01  4e-03  2e-18  3e-16
 8: -1.0140e-01 -1.0283e-01  1e-03  2e-18  4e-16
 9: -1.0182e-01 -1.0226e-01  4e-04  1e-18  4e-16
10: -1.0198e-01 -1.0205e-01  7e-05  8e-19  4e-16
11: -1.0202e-01 -1.0202e-01  3e-06  2e-18  5e-16
12: -1.0202e-01 -1.0202e-01  5e-08  3e-18  5e-16
Optimal solution found.
fold num = 2
     pcost       dcost       gap    pres   dres
 0: -9.6590e+01 -4.3393e+00  9e+03  9e+01  5e-13
 1: -8.8904e+00 -1.5352e+00  5e+02  5e+00  4e-13
 2: -9.8509e-01 -4.8544e-01  3e+01  3e-01  3e-14
 3: -2.1396e-01 -3.3838e-01  3e+00  2e-02  4e-15
 4: -8.86

In [18]:
for item in accuracy_dict :
    print(f"{item} = {accuracy_dict[item]}")

for C = 0.0001 avarage accuracy = 0.8883399810066477
for C = 0.001 avarage accuracy = 0.9475707502374169
for C = 0.01 avarage accuracy = 0.9623741690408357
for C = 0.1 avarage accuracy = 0.9623798670465338
for C = 10 avarage accuracy = 0.9432554605887941
for C = 100 avarage accuracy = 0.9432554605887941
for C = 1000 avarage accuracy = 0.9432554605887941
for C = 10000 avarage accuracy = 0.9432554605887941


In [19]:
print(f"thus best C is {best_c}")

thus best C is 0.1


In [20]:
model = SoftSVM(C=best_c)

In [21]:
model.fit(X_trn, y_trn)

     pcost       dcost       gap    pres   dres
 0: -1.3587e+02 -3.3077e+02  1e+04  3e+01  6e-13
 1: -2.8142e+01 -3.0352e+02  9e+02  1e+00  3e-13
 2: -1.7180e+01 -1.5666e+02  2e+02  3e-01  9e-14
 3: -1.1167e+01 -5.7964e+01  7e+01  8e-02  4e-14
 4: -8.9801e+00 -2.1149e+01  2e+01  1e-02  2e-14
 5: -9.4961e+00 -1.3510e+01  5e+00  3e-03  2e-14
 6: -1.0047e+01 -1.1504e+01  2e+00  4e-04  3e-14
 7: -1.0413e+01 -1.0710e+01  3e-01  6e-06  3e-14
 8: -1.0517e+01 -1.0562e+01  5e-02  9e-07  3e-14
 9: -1.0535e+01 -1.0537e+01  2e-03  2e-08  4e-14
10: -1.0536e+01 -1.0536e+01  6e-05  5e-10  4e-14
11: -1.0536e+01 -1.0536e+01  1e-06  8e-12  4e-14
Optimal solution found.


In [22]:
predictions = model.predict(X_trn)
accuracy = sum(predictions == y_trn) / len(y_trn)
print(f" model Accuracy = {accuracy:.2f} on 'train' data")

 model Accuracy = 0.99 on 'train' data


In [23]:
predictions = model.predict(X_tst)
accuracy = sum(predictions == y_tst) / len(y_tst)
print(f" model Accuracy = {accuracy:.2f} on 'test' data")

 model Accuracy = 0.97 on 'test' data


# Part C

In [50]:
# training all n(n-1)/2 combination
models_dic = {}
for i in range(0, 10):
    for j in range(i+1, 10):     
        print(f"between {i} and {j}")

        Train_mask = (y_train == i) | (y_train == j)
        X_trn = X_train[Train_mask]
        y_trn = BinaryLabel(y_train[Train_mask], i)

        models_dic[f"{i}_{j}"] = SoftSVM(C=0.1)
        models_dic[f"{i}_{j}"] .fit(X_trn, y_trn)

between 0 and 1
     pcost       dcost       gap    pres   dres
 0: -1.8051e+01 -2.3601e+02  7e+03  2e+01  1e-13
 1: -4.2686e+00 -2.0839e+02  5e+02  8e-01  1e-13
 2: -2.6234e+00 -8.1030e+01  1e+02  2e-01  3e-14
 3: -1.3004e+00 -3.1532e+01  5e+01  7e-02  1e-14
 4: -5.0096e-01 -1.1494e+01  2e+01  2e-02  8e-15
 5: -2.8155e-01 -4.1557e+00  5e+00  7e-03  5e-15
 6: -2.4518e-01 -1.1287e+00  1e+00  1e-03  5e-15
 7: -2.5344e-01 -7.3526e-01  5e-01  2e-04  5e-15
 8: -3.3986e-01 -4.8180e-01  1e-01  4e-05  5e-15
 9: -3.6122e-01 -4.2669e-01  7e-02  2e-16  6e-15
10: -3.7962e-01 -3.9978e-01  2e-02  2e-16  5e-15
11: -3.8664e-01 -3.8965e-01  3e-03  2e-16  6e-15
12: -3.8804e-01 -3.8808e-01  4e-05  2e-16  6e-15
13: -3.8806e-01 -3.8806e-01  4e-07  2e-16  6e-15
14: -3.8806e-01 -3.8806e-01  4e-09  2e-16  6e-15
Optimal solution found.
between 0 and 2
     pcost       dcost       gap    pres   dres
 0: -7.0486e+01 -2.8335e+02  1e+04  2e+01  3e-13
 1: -1.4776e+01 -2.5643e+02  7e+02  1e+00  3e-13
 2: -8.3723e+00

In [51]:
# implementing OneVsOne classification
X = X_test
y = y_test

pred_candid = np.zeros((y.shape[0], 10))

for i in range(0, 3):
    for j in range(i+1, 3):
        
        pred_i_j = models_dic[f"{i}_{j}"].predict(X)
        
        for idx, prediction in enumerate(pred_i_j) :
            if prediction == 1:
                pred_candid[idx, i] += 1
            else :
                pred_candid[idx, j] += 1
                        

In [59]:
accuracy = sum(pred_candid.argmax(axis=1)==y_test) / len(y_test)
print( f"OneVsOne accuracy on test data = {accuracy}")

OneVsOne accuracy on test data = 0.299


In [64]:
a = np.random.randint(0, 10, y_test.shape)
accuracy = sum((a==y_test)) / len(y_test)
print( f"Random Labelling accuracy on test data = {accuracy}")

Random Labelling accuracy on test data = 0.1085


# Part D

In [26]:
class G_SoftSVM(object):
    def __init__(self, C=5, sigma=1):
        
        self.sigma = sigma
        self.C = C
        self.alpha = None
        self.SV = None
        self.SV_label = None
  

    def fit(self, X, y):
        n_samples, n_features = X.shape
        K = np.array([[np.exp(-np.linalg.norm(X[i]-X[j])**2 / (2 * (self.sigma ** 2))) for j in range(n_samples)] for i in range(n_samples)])
        P = 1.0 * cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = 1.0 * cvxopt.matrix(y, (1,n_samples))
        b = cvxopt.matrix(0.0)
        G = cvxopt.matrix(np.vstack((np.diag(np.ones(n_samples) * -1), np.identity(n_samples))))
        h = cvxopt.matrix(np.hstack((np.zeros(n_samples), np.ones(n_samples) * self.C)))
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        alpha = np.ravel(solution['x'])
        
        
        SV = alpha > 1e-5
        
       
        self.alpha = alpha[SV]
        self.SV = X[SV]
        self.SV_label = y[SV]
        
            
        ind = np.arange(len(alpha))[SV]
        self.b = 0
        for n in range(len(self.alpha)):
            self.b += self.SV_label[n]
            self.b -= np.sum(self.alpha * self.SV_label * K[ind[n],SV])
        self.b /= len(self.alpha)


    def predict(self, X):
        y_predict = np.zeros(len(X))
        for i in range(len(X)):
            s = 0
            
         
            
            for alpha, SV_label, SV in zip(self.alpha, self.SV_label, self.SV):
                s += alpha * SV_label * np.exp(-np.linalg.norm(X[i]-SV)**2 / (2 * (self.sigma ** 2)))
            y_predict[i] = s
        res = y_predict + self.b

        return np.sign(res)

In [27]:
from sklearn.model_selection import KFold
best_c = 1
best_sigma = 0
best_accuracy = 0

accuracy_dict = {}

for C  in [ 0.01, 0.1, 1, 10, 100]:
    sigma = 0.5
    for sigma in [0.01*sigma, 0.1*sigma, 2*sigma, 10*sigma]:
        
        print(f"C is {C} and sigma is {sigma}")

        model = G_SoftSVM(C=C, sigma=sigma)


        num_folds = 5
        
        X = X_trn
        y = y_trn


        kfold = KFold(n_splits=num_folds)


        accuracies = []

        fold_num = 0
        for train, test in kfold.split(X):
            fold_num += 1
            print(f"fold num = {fold_num}")

            CV_X_train, CV_X_test, CV_y_train, CV_y_test = X[train], X[test], y[train], y[test]

            model.fit(CV_X_train, CV_y_train)

            predictions = model.predict(CV_X_test)
            accuracy = sum(predictions == CV_y_test) / len(CV_y_test)
            accuracies.append(accuracy)
        



        avg_accuracy = np.mean(accuracies)

        print(f"for C = {C} and sigma = {sigma} avarage accuracy = {avg_accuracy}")

        accuracy_dict[f"for C = {C} and sigma = {sigma} avarage accuracy"] = avg_accuracy

        if avg_accuracy > best_accuracy :
            best_accuracy = avg_accuracy
            best_c = C
            best_sigma = sigma
                

C is 0.01 and sigma is 0.005
fold num = 1
     pcost       dcost       gap    pres   dres
 0: -3.5499e+02 -9.2989e+01  4e+03  7e+01  3e-16
 1: -5.5651e+01 -2.5118e+01  1e+02  2e+00  4e-16
 2: -1.1970e+01 -2.1995e+01  1e+01  2e-14  9e-16
 3: -1.2450e+01 -1.2664e+01  2e-01  2e-15  4e-16
 4: -1.2472e+01 -1.2485e+01  1e-02  6e-16  1e-16
 5: -1.2478e+01 -1.2478e+01  1e-04  6e-17  3e-16
 6: -1.2478e+01 -1.2478e+01  1e-06  5e-17  3e-16
Optimal solution found.
fold num = 2
     pcost       dcost       gap    pres   dres
 0: -3.5502e+02 -9.2975e+01  4e+03  7e+01  3e-16
 1: -5.5577e+01 -2.5096e+01  1e+02  2e+00  3e-16
 2: -1.1985e+01 -2.1972e+01  1e+01  1e-15  7e-16
 3: -1.2476e+01 -1.2694e+01  2e-01  7e-17  3e-16
 4: -1.2537e+01 -1.2550e+01  1e-02  8e-17  5e-16
 5: -1.2537e+01 -1.2538e+01  1e-04  1e-16  2e-16
 6: -1.2537e+01 -1.2537e+01  1e-06  8e-16  3e-16
Optimal solution found.
fold num = 3
     pcost       dcost       gap    pres   dres
 0: -3.5486e+02 -9.3009e+01  4e+03  7e+01  0e+00
 1: -

In [28]:
for item in accuracy_dict:
    print(f"{item} = {accuracy_dict[item]}")

for C = 0.01 and sigma = 0.005 avarage accuracy = 0.5066246056782334
for C = 0.01 and sigma = 0.05 avarage accuracy = 0.5066246056782334
for C = 0.01 and sigma = 1.0 avarage accuracy = 0.5066246056782334
for C = 0.01 and sigma = 5.0 avarage accuracy = 0.9558359621451104
for C = 0.1 and sigma = 0.005 avarage accuracy = 0.5066246056782334
for C = 0.1 and sigma = 0.05 avarage accuracy = 0.5066246056782334
for C = 0.1 and sigma = 1.0 avarage accuracy = 0.5072555205047318
for C = 0.1 and sigma = 5.0 avarage accuracy = 0.9753943217665615
for C = 1 and sigma = 0.005 avarage accuracy = 0.5066246056782334
for C = 1 and sigma = 0.05 avarage accuracy = 0.5066246056782334
for C = 1 and sigma = 1.0 avarage accuracy = 0.5261829652996846
for C = 1 and sigma = 5.0 avarage accuracy = 0.9854889589905362
for C = 10 and sigma = 0.005 avarage accuracy = 0.5066246056782334
for C = 10 and sigma = 0.05 avarage accuracy = 0.5066246056782334
for C = 10 and sigma = 1.0 avarage accuracy = 0.5261829652996846
for C

In [29]:
print(f"thus best c = {best_c} and best sigma = {best_sigma}")

thus best c = 10 and best sigma = 5.0


In [30]:
G_model = G_SoftSVM(C=best_c, sigma=best_sigma)

In [31]:
G_model.fit(X_trn, y_trn)

     pcost       dcost       gap    pres   dres
 0:  2.0793e+03 -5.6197e+04  1e+05  4e-01  9e-15
 1:  1.9745e+03 -9.1791e+03  1e+04  2e-02  8e-15
 2:  5.0788e+02 -2.1168e+03  3e+03  3e-03  7e-15
 3:  5.1709e+00 -4.9817e+02  5e+02  3e-14  4e-15
 4: -7.5000e+01 -2.1645e+02  1e+02  9e-16  3e-15
 5: -9.7129e+01 -1.4364e+02  5e+01  8e-15  2e-15
 6: -1.0655e+02 -1.2032e+02  1e+01  4e-15  2e-15
 7: -1.0979e+02 -1.1317e+02  3e+00  6e-15  2e-15
 8: -1.1081e+02 -1.1114e+02  3e-01  5e-15  2e-15
 9: -1.1093e+02 -1.1095e+02  2e-02  4e-15  2e-15
10: -1.1094e+02 -1.1094e+02  3e-04  6e-15  2e-15
11: -1.1094e+02 -1.1094e+02  7e-06  4e-15  2e-15
Optimal solution found.


In [32]:
predictions = G_model.predict(X_trn)
accuracy = sum(predictions == y_trn) / len(y_trn)
print(f" model Accuracy = {accuracy:.2f}  on 'train' data")

 model Accuracy = 1.00  on 'train' data


In [33]:
predictions = model.predict(X_tst)
accuracy = sum(predictions == y_tst) / len(y_tst)
print(f" model Accuracy = {accuracy:.2f}  on 'test' data")

 model Accuracy = 0.99  on 'test' data
